### Text Mining Assignment 

#### 2nd Problem Statement

Extract reviews of any product from ecommerce website like amazon & Perform emotion mining

### Web Scarping Amazon Site

In [1]:
# import module
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")  #--to ignore warnings

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

# user define function
# Scrape the data
def getdata(url):
    r = requests.get(url, headers=HEADERS)
    return r.text


def html_code(url):

    # pass the url
    # into getdata function
    htmldata = getdata(url)
    soup = BeautifulSoup(htmldata, 'html.parser')

    #display html code
    return (soup)


url = "https://www.amazon.in/dp/B09G9G9C2K/ref=redir_mobile_desktop?_encoding=\
    UTF8&aaxitk=c5ff115c1ba1d9f7edec86bc8c6a4f99&hsa_cr_id=9638012290402&pd_rd_plhdr=t&pd_rd_r=\
    afaec463-9f0f-4512-b2d3-240fa990ad1d&pd_rd_w=7msXL&pd_rd_wg=2F2hi&ref_=sbx_be_s_sparkle_mcd_asin_0_img&th=1"

soup = html_code(url)

In [2]:
#Getting Customer Names

def cus_data(soup):
# find the Html tag
# with find()
# and convert into string
    data_str = ""
    cus_list = []

    for item in soup.find_all("span", class_="a-profile-name"):
        data_str = data_str + item.get_text()
        cus_list.append(data_str)
        data_str = ""
    return cus_list

#Listing the Customers
cus_res = cus_data(soup)

In [3]:
#Getting Reviews
def cus_rev(soup):
    # find the Html tag
    # with find()
    # and convert into string
    data_str = ""

    #To Get Review from this Tag
    for item in soup.find_all("div",class_="a-expander-content reviewText review-text-content a-expander-partial-collapse-content"):
        data_str = data_str + item.get_text()

    result = data_str.split("\n")
    return (result)

rev_data = cus_rev(soup)
rev_result = []
for i in rev_data:
    if i is "":
        pass
    else:
        rev_result.append(i) #appending the List of Reviews

In [4]:
#To Save the Review in CSV
import pandas as pd

# initialise data of lists.
data = {'Name': cus_res,
        'review': rev_result}

# Create DataFrame
df = pd.DataFrame(data)

# Save the output.
df.to_csv('C:/Users/Akaash/Downloads/amazon_review.csv')

Inference: Did Web-Scraping And Saved the Data to the local Directory Now Will Do Emotion mining

### Named Entity Recognition (NER)

In [5]:
#Importing the Required Library
import numpy as np 
import pandas as pd 
import string 
import spacy 

from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
#loading The Dataset
reviews=pd.read_csv("C:/Users/Akaash/Downloads/amazon_review.csv", encoding = 'Latin-1')
reviews.drop(['Unnamed: 0'],inplace = True, axis = 1)
reviews

,Name,review
0,Anil,Its really Shocking that there is no charger...
1,Jofert,"Like iphone 11,12 now 13. All same look and ..."
2,Vempati Harish Chakravarthi,There are a bunch of reviews on the product ...
3,Nitin Verma,Same as previous model iPhone 12 with smalle...
4,Brajesh Bharti,I would suggest 12 Pro over 13 mini. Just a ...
5,Ishaan Aditya,I've been using the iPhone 13 mini for more ...
6,Joshuaa Anand,Writing from 'iPhone xs max' I dint see a RE...
7,Arvind,Appleâs another generation phone with noth...


#### Pre - Processing

In [7]:
# remove both the leading and the trailing characters
reviews = [review.strip() for review in reviews.review] 
# removes empty strings, because they are considered in Python as False
reviews = [review for review in reviews if review] 
reviews[0:3]

["Its really Shocking that there is no charger which is totally a scam with the customers by Apple they think Indians are Fool and some Indians  are really doing foolish by purchasing this Iphone without charger...don't buy Iphone until they  provide charger total boycott this Iphone...",
 'Like iphone 11,12 now 13. All same look and minor differences. Why Apple fellows release phone with no new features',
 'There are a bunch of reviews on the product already. Just sharing my 1st day experience so far, I loved it. This is a major upgrade for me, I jumped from iPhone 7 to 13mini after 4 years. Always wanted to smaller iPhone and looks like so far 13mini is by far the best predecessor of iPhone 7( for me interns of size vs performance ratio). Fantastic Camera, better battery life, and incredibly light phone. All the faults from iPhone 12mini are addressed in this phone. This will be a model which I can hold for at-least next three years in my opinion.']

In [8]:
#Joining the list into one string/text
review_text = ' '.join(reviews)

#### POS Tagging

In [ ]:
##Part Of Speech Tagging
nlp = spacy.load('en_core_web_sm')

#Taking a Part of the Corpus and Tagging is POS
one_block = review_text
doc_block = nlp(one_block)

#Tokenization with POS Tagged
for token in doc_block[0:20]:
    print(token, token.pos_)

Inference: Parts of Speech for the first 20 words

In [ ]:
#Filtering for nouns and verbs only
nouns_verbs = [token.text for token in doc_block if token.pos_ in ('NOUN', 'VERB')]
print(nouns_verbs[0:50])

Inference: Noun & Verbs from first 50 Words

In [ ]:
#Counting tokens again
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

X = cv.fit_transform(nouns_verbs)
sum_words = X.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

wf_df = pd.DataFrame(words_freq)
wf_df.columns = ['word', 'count']

#Top 10
wf_df[0:10]

Inference: Top 10 Words/Token of Noun & Verbs

In [ ]:
##Visualizing results
#Barchart for top 10 nouns + verbs
wf_df[0:10].plot.bar(x='word', figsize=(12,8), title='Top verbs and nouns')

Inference: We Can See the Count of Each Words in the Bar Chart

### Emotion Mining / Sentiment Analysis

In [ ]:
#Taking Each Tweets as one Token(Sentence)
from nltk import tokenize
sentences = tokenize.sent_tokenize(" ".join(reviews))
sentences[0:5]

In [ ]:
#Creating a Dataframe of Sentences
sent_df = pd.DataFrame(sentences, columns=['sentence'])
sent_df

Inference: We Have Total 76 Sentences

In [ ]:
#Sentiment analysis - Loading Afinn Lexicon
afinn = pd.read_csv('C:/Users/Akaash/Downloads/Afinn.csv', sep=',', encoding='latin-1')
afinn.shape

Inference: Loaded the afinn Lexicon for emotion mining

In [ ]:
# Taking the affinity_scores from lexicon into Dictionary
affinity_scores = afinn.set_index('word')['value'].to_dict()

In [ ]:
#Custom function :score each word in a sentence in lemmatised form, 
#but calculate the score for the whole original sentence.
nlp = spacy.load('en_core_web_sm')
sentiment_lexicon = affinity_scores

def calculate_sentiment(text: str = None):
    sent_score = 0
    if text:
        sentence = nlp(text)
        for word in sentence:
            sent_score += sentiment_lexicon.get(word.lemma_, 0)
    return sent_score

In [ ]:
#Calling the function 
calculate_sentiment(text = 'worst')

Inference: User Defined Function Words Properly

In [ ]:
#Applying to the Whole DataFrame
sent_df['sentiment_value'] = sent_df['sentence'].apply(calculate_sentiment)

# how many words are in the sentence? (Adding Wordcount column)
sent_df['word_count'] = sent_df['sentence'].str.split().apply(len)
sent_df['word_count'].head()

#### Descriptive Stats on the affinity scores

In [ ]:
# Sentiment score of the whole review
sent_df['sentiment_value'].describe()

Inferences : Max Score is 6, Min Score is -7

In [ ]:
# Sentiment score of the review - Negative Score Filtering
sent_df[sent_df['sentiment_value']<0].head()

In [ ]:
# Sentiment score of the review - Positive Score Filtering
sent_df[sent_df['sentiment_value']>0].head()

In [ ]:
# Sentiment score of the review - Lowest Score Filtering
sent_df[sent_df['sentiment_value']<-6].head()

In [ ]:
# Sentiment score of the review - Highest ScoreFiltering
sent_df[sent_df['sentiment_value']>5].head()

In [ ]:
#Adding A index Column
sent_df['index']=range(0,len(sent_df))
sent_df.head()

#### Visualization of Sentiment / Emotion

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(sent_df['sentiment_value'])

Inference: It is Left Skewed, thus Making the reviews more of Negative Kind

In [ ]:
#Line Chart
plt.figure(figsize=(15, 10))
sns.lineplot(y='sentiment_value',x='index',data=sent_df)

Inference: Here We Can say the Tweets mostly Comes under the range of -2 to 2 which can be Called as a Neutral, with Some Positive and Negative spikes in between

In [ ]:
#Scatterplot
sent_df.plot.scatter(x='word_count', y='sentiment_value', figsize=(8,8), 
                     title='Sentence sentiment value to sentence word count')

Inference: From this Plot, it can be seen it does not really matter word count to have good sentiment_value or bad sentiment value as it can be seen, No Relationship betwwen Sentiment_value & word_count, And we can say this Corpus has a Neutral Polarity here